# Configure AlloyDB AI Natural Language

> IMPORTANT: This is the fourth notebook in the lab. The notebooks build on top of each other, so be sure to run the preceding notebooks, in order, before running this one. Start your journey building ADK Agents with MCP Toolbox [here](./1_setup_and_explore_databases.ipynb). 

## Overview

This notebook introduces you to AlloyDB AI Natural Language, a powerful new way to interact with databases. You will learn how to integrate this feature into your database and leverage it as an MCP Toolbox tool to build a more intuitive and flexible ADK agent. The key milestones you will achieve are:

- Configuring AlloyDB AI Natural Language: You will walk through the process of setting up AlloyDB AI's natural language capabilities on your finance database. This includes enabling the necessary extensions, creating a natural language configuration, registering tables, and generating context to help the model understand your schema.
- Deploying an Advanced Tool to MCP Toolbox: You will create and deploy an MCP Toolbox tool that allows your agent to translate natural language questions into SQL queries, enabling more dynamic and flexible interactions with the database.
- Integrating the Natural Language Tool with the ADK Agent: You will update your deployed ADK agent to use the new natural language tool. This will demonstrate how to build an agent that can understand and respond to a wide range of user queries without needing predefined, specific tools for every possible question.
- Testing the Enhanced Agent: You will interact with the updated agent to see the AlloyDB natural language feature in action. This will highlight how the agent can now handle more complex and varied questions about your financial data, showcasing the power of combining ADK, MCP Toolbox, and AlloyDB AI.

### Terraform Resources

The following pre-requisite resources were created for you by Terraform. See the [main.tf](../terraform/main.tf) file for more details on the environment configuration:

- AlloyDB: The AlloyDB cluster and instance that you will configure with the natural language feature.
- Custom VPC (demo-vpc): The private network that ensures secure communication between the ADK agent, MCP Toolbox, and the AlloyDB instance.
- Cloud Run: Hosts the ADK agent and MCP Toolbox as secure, scalable services within a private VPC.
- Vertex AI Workbench: The Vertex AI Workbench instance where you are executing this notebook.

### Google Cloud Services Used

This notebook utilizes the following Google Cloud services:
- AlloyDB for PostgreSQL: The core database service that you will enhance with the AI natural language feature.
- Vertex AI: The platform that provides the underlying AI and machine learning capabilities for the AlloyDB natural language feature.
- MCP Toolbox for Databases: The open-source tool server that you will update with the new natural language tool.
- Cloud Run: The service that hosts your deployed ADK agent and MCP Toolbox.
- Secret Manager: Used to securely manage your tools.yaml configuration and database credentials.
- IAM (Identity and Access Management): Manages the necessary permissions for the services and service accounts to interact with each other.

### Logical Flow

This notebook is structured to guide you step-by-step through the process of enabling and using the AlloyDB natural language feature:
- Basic Setup: This section prepares your environment by defining variables, connecting to your Google Cloud project, and installing the required Python libraries.
- Integrate AlloyDB with Vertex AI: You will connect to your AlloyDB cluster and grant the necessary permissions for it to integrate with Vertex AI, which is a prerequisite for using the natural language feature.
- Natural Language Setup: This is the core of the notebook, where you will:
    - Enable the alloydb_ai_nl extension.
    - Create a natural language configuration.
    - Register the tables from your finance database with the configuration.
    - Generate and apply context for your tables and columns, which helps the AI model understand your schema.
    - Define query templates to provide the model with examples of valid SQL.
- Generate and Execute SQL with AlloyDB Natural Language: You will test the natural language feature directly in the database by providing natural language questions and observing the generated SQL.
- Use AlloyDB Natural Language with MCP Toolbox: You will update your `tools.yaml` file to include a new tool of kind alloydb-ai-nl, which will allow your agent to leverage the natural language capabilities you just configured.
- Use AlloyDB Natural Language with the ADK Agent: Finally, you will update your deployed ADK agent to use the new tool and interact with it to ask a variety of natural language questions about your financial data.

## Basic Setup

### Define Notebook Variables

Update the `project_id` and `region` variables below to match your environment. You can use defaults for the rest of the project variables. 

You will be prompted for the AlloyDB password you chose then you provisioned the environment with Terraform.

In [ ]:
# Project variables
project_id = "your-project"
region = "your-region"
vpc = "demo-vpc"
gcs_bucket_name = f"project-files-{project_id}"

# AlloyDB variables
alloydb_cluster = "my-alloydb-cluster"
alloydb_instance = "my-alloydb-instance"
alloydb_database = "finance"
alloydb_password = input("Please enter the password for the AlloyDB 'postgres' database user: ")

In [ ]:
# Set env variable to suppress annoying system warnings when running shell commands
%env GRPC_ENABLE_FORK_SUPPORT=1

### Connect to your Google Cloud Project

In [ ]:
# Configure gcloud.
!gcloud config set project {project_id}

### Configure Logging

In [ ]:
import logging
import sys

# Configure the root logger to output messages with INFO level or above
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='%(asctime)s[%(levelname)5s][%(name)14s] - %(message)s',  datefmt='%H:%M:%S', force=True)

### Install Dependencies

In [ ]:
! pip install --quiet google-cloud-storage==2.19.0 \
                      asyncpg==0.30.0 \
                      google.cloud.alloydb.connector==1.9.0 

### Define Helper Functions

#### REST API Helper Function

In [ ]:
import requests
import google.auth
import json

# Get an access token based upon the current user
creds, _ = google.auth.default()
authed_session = google.auth.transport.requests.AuthorizedSession(creds)
access_token=creds.token

if project_id:
  authed_session.headers.update({"x-goog-user-project": project_id}) # Required to workaround a project quota bug

def rest_api_helper(
    url: str,
    http_verb: str,
    request_body: dict = None,
    params: dict = None,
    session: requests.Session = authed_session,
  ) -> dict:
  """Calls a REST API using a pre-authenticated requests Session."""

  headers = {"Content-Type": "application/json"}

  try:

    if http_verb == "GET":
      response = session.get(url, headers=headers, params=params)
    elif http_verb == "POST":
      response = session.post(url, json=request_body, headers=headers, params=params)
    elif http_verb == "PUT":
      response = session.put(url, json=request_body, headers=headers, params=params)
    elif http_verb == "PATCH":
      response = session.patch(url, json=request_body, headers=headers, params=params)
    elif http_verb == "DELETE":
      response = session.delete(url, headers=headers, params=params)
    else:
      raise ValueError(f"Unknown HTTP verb: {http_verb}")

    # Raise an exception for bad status codes (4xx or 5xx)
    response.raise_for_status()

    # Check if response has content before trying to parse JSON
    if response.content:
        return response.json()
    else:
        return {} # Return empty dict for empty responses (like 204 No Content)

  except requests.exceptions.RequestException as e:
      # Catch potential requests library errors (network, timeout, etc.)
      # Log detailed error information
      print(f"Request failed: {e}")
      if e.response is not None:
          print(f"Request URL: {e.request.url}")
          print(f"Request Headers: {e.request.headers}")
          print(f"Request Body: {e.request.body}")
          print(f"Response Status: {e.response.status_code}")
          print(f"Response Text: {e.response.text}")
          # Re-raise a more specific error or a custom one
          raise RuntimeError(f"API call failed with status {e.response.status_code}: {e.response.text}") from e
      else:
          raise RuntimeError(f"API call failed: {e}") from e
  except json.JSONDecodeError as e:
      print(f"Failed to decode JSON response: {e}")
      print(f"Response Text: {response.text}")
      raise RuntimeError(f"Invalid JSON received from API: {response.text}") from e


#### AlloyDB Helper Function

In [ ]:
# Create AlloyDB Query Helper Function
import sqlalchemy
from sqlalchemy import text, exc
import pandas as pd

async def run_alloydb_query(pool, sql: str, params = None, output_as_df: bool = True):
    """Executes a SQL query or statement against the database pool.

    Handles various SQL statements:
    - SELECT/WITH: Returns results as a DataFrame (if output_as_df=True)
      or ResultProxy. Supports parameters. Does not commit.
    - EXPLAIN/EXPLAIN ANALYZE: Executes the explain, returns the query plan
      as a formatted multi-line string. Ignores output_as_df.
      Supports parameters. Does not commit.
    - INSERT/UPDATE/DELETE/CREATE/ALTER etc.: Executes the statement,
      commits the transaction, logs info, and returns the ResultProxy.
      Supports single or bulk parameters (executemany).

    Args:
      pool: An asynchronous SQLAlchemy connection pool.
      sql: A string containing the SQL query or statement template.
      params: Optional.
        - None: Execute raw SQL (Use with caution for non-SELECT/EXPLAIN).
        - dict or tuple: Parameters for a single execution.
        - list of dicts/tuples: Parameters for bulk execution (executemany).
      output_as_df (bool): If True and query is SELECT/WITH, return pandas DataFrame.
                           Ignored for EXPLAIN and non-data-returning statements.

    Returns:
      pandas.DataFrame | str | sqlalchemy.engine.Result | None:
        - DataFrame: For SELECT/WITH if output_as_df=True.
        - str: For EXPLAIN/EXPLAIN ANALYZE, containing the formatted query plan.
        - ResultProxy: For non-SELECT/WITH/EXPLAIN statements, or SELECT/WITH
                       if output_as_df=False.
        - None: If a SQLAlchemy ProgrammingError or other specific error occurs.

    Raises:
        Exception: Catches and logs `sqlalchemy.exc.ProgrammingError`, returning None.
                   May re-raise other database exceptions.

    Example Execution:
      Single SELECT:
        sql_select = "SELECT ticker, company_name from investments LIMIT 5"
        df_result = await run_alloydb_query(pool, sql_select)

      Single non-SELECT - Parameterized (Safe!):
        Parameterized INSERT:
          sql_insert = "INSERT INTO investments (ticker, company_name) VALUES (:ticker, :name)"
          params_insert = {"ticker": "NEW", "name": "New Company"}
          insert_result = await run_alloydb_query(pool, sql_insert, params_insert)

        Parameterized UPDATE:
          sql_update = "UPDATE products SET price = :price WHERE id = :product_id"
          params_update = {"price": 99.99, "product_id": 123}
          update_result = await run_alloydb_query(pool, sql_update, params_update)

      Bulk Update:
        docs = pd.DataFrame([
            {'id': 101, 'sparse_embedding': '[0.1, 0.2]'},
            {'id': 102, 'sparse_embedding': '[0.3, 0.4]'},
            # ... more rows
        ])

        update_sql_template = '''
            UPDATE products
            SET sparse_embedding = :embedding,
                sparse_embedding_model = 'BM25'
            WHERE id = :product_id
        ''' # Using named parameters :param_name

        # Prepare list of dictionaries for params
        data_to_update = [
            {"embedding": row.sparse_embedding, "product_id": row.id}
            for row in docs.itertuples(index=False)
        ]

        if data_to_update:
          bulk_result = await run_alloydb_query(pool, update_sql_template, data_to_update)
          # bulk_result is the SQLAlchemy ResultProxy

    """
    sql_lower_stripped = sql.strip().lower()
    is_select_with = sql_lower_stripped.startswith(('select', 'with'))
    is_explain = sql_lower_stripped.startswith('explain')

    # Determine if the statement is expected to return data rows or a plan
    is_data_returning = is_select_with or is_explain

    # Determine actual DataFrame output eligibility (only for SELECT/WITH)
    effective_output_as_df = output_as_df and is_select_with

    # Check if params suggest a bulk operation (for logging purposes)
    is_bulk_operation = isinstance(params, (list, tuple)) and len(params) > 0 and isinstance(params[0], (dict, tuple, list))

    async with pool.connect() as conn:
        try:
          # Execute with or without params
          if params:
              result = await conn.execute(text(sql), params)
          else:
              # Add warning for raw SQL only if it's NOT data-returning
              #if not is_data_returning:
                  #logging.warning("Executing non-SELECT/EXPLAIN raw SQL without parameters. Ensure SQL is safe.")
              result = await conn.execute(text(sql))

          # --- Handle statements that return data or plan ---
          if is_data_returning:
              if is_explain:
                  # Fetch and format EXPLAIN output as a string
                    try:
                        plan_rows = result.fetchall()
                        # EXPLAIN output is usually text in the first column
                        query_plan = "\n".join([str(row[0]) for row in plan_rows])
                        #logging.info(f"EXPLAIN executed successfully for: {sql[:100]}...")
                        return query_plan
                    except Exception as e:
                        logging.error(f"Error fetching/formatting EXPLAIN result: {e}")
                        return None
              else: # Handle SELECT / WITH
                  if effective_output_as_df:
                      try:
                          rows = result.fetchall()
                          column_names = result.keys()
                          df = pd.DataFrame(rows, columns=column_names)
                          #logging.info(f"SELECT/WITH executed successfully, returning DataFrame for: {sql[:100]}...")
                          return df
                      except Exception as e:
                          logging.error(f"Error converting SELECT result to DataFrame: {e}")
                          logging.info(f"Returning raw ResultProxy for SELECT/WITH due to DataFrame conversion error for: {sql[:100]}...")
                          return result # Fallback to raw result
                  else:
                      # Return raw result proxy for SELECT/WITH if df output not requested
                      #logging.info(f"SELECT/WITH executed successfully, returning ResultProxy for: {sql[:100]}...")
                      return result

          # --- Handle Non-Data Returning Statements (INSERT, UPDATE, DELETE, CREATE, etc.) ---
          else:
              await conn.commit() # Commit changes ONLY for these statements
              operation_type = sql.strip().split()[0].upper()
              row_count = result.rowcount # Note: rowcount behavior varies

              if is_bulk_operation:
                  print(f"Bulk {operation_type} executed for {len(params)} items. Result rowcount: {row_count}")
              elif operation_type in ['INSERT', 'UPDATE', 'DELETE']:
                  print(f"{operation_type} statement executed successfully. {row_count} row(s) affected.")
              else: # CREATE, ALTER, etc.
                  print(f"{operation_type} statement executed successfully. Result rowcount: {row_count}")
              return result # Return the result proxy

        except exc.ProgrammingError as e:
            # Log the error with context
            logging.error(f"SQL Programming Error executing query:\nSQL: {sql[:500]}...\nParams (sample): {str(params)[:500]}...\nError: {e}")
            # Rollback might happen automatically on context exit with error, but explicit can be clearer
            # await conn.rollback() # Consider if needed based on pool/transaction settings
            return None # Return None on handled programming errors
        except Exception as e:
            # Log other unexpected errors
            logging.error(f"An unexpected error occurred during query execution:\nSQL: {sql[:500]}...\nError: {e}")
            # await conn.rollback() # Consider if needed
            raise # Re-raise unexpected errors



## Integrate AlloyDB with Vertex AI

### Connect to the AlloyDB Cluster

This function will create a connection pool to your AlloyDB instance using the AlloyDB Python connector. The AlloyDB Python connector will automatically create secure connections to your AlloyDB instance using mTLS.

In [ ]:
import asyncpg

import sqlalchemy
from sqlalchemy.ext.asyncio import AsyncEngine, create_async_engine

from google.cloud.alloydb.connector import AsyncConnector, IPTypes

async def init_connection_pool(connector: AsyncConnector, db_name: str = alloydb_database, pool_size: int = 5) -> AsyncEngine:
    # initialize Connector object for connections to AlloyDB
    connection_string = f"projects/{project_id}/locations/{region}/clusters/{alloydb_cluster}/instances/{alloydb_instance}"

    async def getconn() -> asyncpg.Connection:
        conn: asyncpg.Connection = await connector.connect(
            connection_string,
            "asyncpg",
            user="postgres",
            password=alloydb_password,
            db=db_name,
            ip_type=IPTypes.PRIVATE, # Optionally use IPTypes.PUBLIC
        )
        return conn

    pool = create_async_engine(
        "postgresql+asyncpg://",
        async_creator=getconn,
        pool_size=pool_size,
        max_overflow=0,
        isolation_level='AUTOCOMMIT'
    )
    return pool

connector = AsyncConnector()

finance_db_pool = await init_connection_pool(connector, f"{alloydb_database}")

### Grant Vertex AI Permission to AlloyDB Service Account

In [ ]:
# Get project number
project_number = ! gcloud projects describe {project_id} --format='value(projectNumber)'
project_number = project_number[0]

# Add Vertex AI Permissions
alloydb_service_account = f"serviceAccount:service-{project_number}@gcp-sa-alloydb.iam.gserviceaccount.com"
!gcloud projects add-iam-policy-binding {project_id} \
    --member="{alloydb_service_account}" \
    --role="roles/aiplatform.user"
  

## Natural Language Setup

### Install Required Extensions

In [ ]:
# Enables Vertex AI integration
sql = "CREATE EXTENSION IF NOT EXISTS google_ml_integration;"
await run_alloydb_query(finance_db_pool, sql)

In [ ]:
# Enables AlloyDB AI NL feature
sql = "CREATE EXTENSION IF NOT EXISTS alloydb_ai_nl cascade;"
await run_alloydb_query(finance_db_pool, sql)

In [ ]:
# Enables Parameterized Secure View feature (required to run AI NL tool on MCP Toolbox)
sql = "CREATE EXTENSION IF NOT EXISTS parameterized_views cascade;"
await run_alloydb_query(finance_db_pool, sql)

In [ ]:
# Verify the extensions are installed
sql = "SELECT extversion FROM pg_extension WHERE extname IN ('google_ml_integration','alloydb_ai_nl','parameterized_views');"
await run_alloydb_query(finance_db_pool, sql)

### Create a Natural Language Configuration

This creates a configuration that will group together the tables, context, and templates for our natural language agent. Success will be shown as `g_create_configuration = None`.

In [ ]:
nl_config = "finance_agent_config"
sql = f"SELECT alloydb_ai_nl.g_create_configuration( '{nl_config}' );"
await run_alloydb_query(finance_db_pool, sql)

### Register Tables to the `finance_agent_config` Config

This step tells our natural language configuration which tables it should be aware of when generating SQL queries. Success will be shown as `g_manage_configuration = True`. 

In [ ]:
nl_config = "finance_agent_config"
sql = f"""
SELECT alloydb_ai_nl.g_manage_configuration(
    operation => 'register_table_view',
    configuration_id_in => '{nl_config}',
    table_views_in=>'{{public.transactions, public.cards, public.users, public.mcc_codes, public.fraud_labels}}'
);
"""
print(sql)
await run_alloydb_query(finance_db_pool, sql)

### Create and Apply Context for Table and Columns

Here, we automatically generate descriptions for our tables and columns. This context is crucial for helping the AI model understand the meaning of our schema. Success will be shown as `generate_schema_context = None`.

In [ ]:
# Generate schema contexts for the tables and their columns that are registered in the nla_demo_cfg configuration
# This query opulates the alloydb_ai_nl.generated_schema_context_view view with context. Passing TRUE overwrites 
# the context in this view from previous runs.

sql = f"""
SELECT alloydb_ai_nl.generate_schema_context(
  '{nl_config}',
  TRUE
);
"""
await run_alloydb_query(finance_db_pool, sql)

In [ ]:
# Verify the generated context
sql = f"""
SELECT *
FROM alloydb_ai_nl.generated_schema_context_view;
"""
await run_alloydb_query(finance_db_pool, sql)

You can optionally modify the generated context using commands like the following:

```
SELECT alloydb_ai_nl.update_generated_relation_context(
  'nla_demo.products',
  'The "nla_demo.products" table stores product details such as ID, name, description, brand, category linkage, and record creation time.'
);

SELECT alloydb_ai_nl.update_generated_column_context(
  'nla_demo.products.name',
  'The "name" column in the "nla_demo.products" table contains the specific name or title of each product.'
);
```

In [ ]:
# Apply the generated context

sql = f"""
SELECT *
FROM alloydb_ai_nl.generated_schema_context_view;
"""
result = await run_alloydb_query(finance_db_pool, sql)

for index, row in result.iterrows():
    if row['schema_object'].count(".") == 1:
        sql = f"""
        SELECT alloydb_ai_nl.apply_generated_relation_context(
          '{row['schema_object']}', true
        );
        """
    else:
        sql = f"""
        SELECT alloydb_ai_nl.apply_generated_column_context(
          '{row['schema_object']}', true
        );
        """
    result = await run_alloydb_query(finance_db_pool, sql)
    print(result)

In [ ]:
# Verify context has all been applied - should return an empty result
sql = f"""
SELECT *
FROM alloydb_ai_nl.generated_schema_context_view;
"""
await run_alloydb_query(finance_db_pool, sql)

### Define Query Templates

Query templates provide the model with examples of well-formed SQL and natural language queries, which improves the accuracy of the generated queries.

In [ ]:
sql_array = []

sql_array.append(f"""
SELECT alloydb_ai_nl.add_template(
    nl_config_id => '{nl_config}',
    intent => 'Are there any recent transactions where a chip card was swiped?',
    sql => 'SELECT t.id AS transaction_id, t.date, t.amount, c.card_number, u.id AS user_id, t.merchant_city, t.merchant_state FROM transactions t JOIN cards c ON t.card_id = c.id JOIN users u ON t.client_id = u.id WHERE c.has_chip = TRUE AND t.use_chip <> ''Chip''',
    sql_explanation => 'This query identifies transactions where a card equipped with a chip was used in a way that didn''t utilize the chip (e.g. swiped). This can be an indicator of card skimming or cloning, as fraudsters may create a magnetic stripe copy of a chip card.'
);
""")

sql_array.append(f"""
SELECT alloydb_ai_nl.add_template(
    nl_config_id => '{nl_config}',
    intent => 'Show me recent occurrences of high value transactions for customers with low income.',
    sql => 'SELECT u.id AS user_id, u.yearly_income, t.id AS transaction_id, t.amount, t.date, mc.description AS merchant_category FROM users u JOIN transactions t ON u.id = t.client_id JOIN mcc_codes mc ON t.mcc = mc.mcc WHERE u.yearly_income < ''30000'' AND t.amount > 1000.00 ORDER BY t.amount DESC',
    sql_explanation => 'This query looks for transactions that are significantly larger than what might be expected from a user''s reported income. This could indicate account takeover or identity theft. For this example, we could define a high-value transaction as one over $1,000 for users with a yearly income of less than $30,000.'
);
""")

sql_array.append(f"""
SELECT alloydb_ai_nl.add_template(
    nl_config_id => '{nl_config}',
    intent => 'Show me transactions that could not feasibly occur so quickly in different locations.',
    sql => 'WITH RankedTransactions AS (SELECT client_id, id AS transaction_id, date, merchant_city, LAG(date, 1) OVER (PARTITION BY client_id ORDER BY date) AS previous_transaction_date, LAG(merchant_city, 1) OVER (PARTITION BY client_id ORDER BY date) AS previous_merchant_city FROM transactions) SELECT rt.client_id, rt.transaction_id, rt.date, rt.merchant_city, rt.previous_transaction_date, rt.previous_merchant_city FROM RankedTransactions rt WHERE rt.previous_transaction_date IS NOT NULL AND rt.merchant_city <> rt.previous_merchant_city AND rt.date - rt.previous_transaction_date < INTERVAL ''1 hour'' ORDER BY rt.client_id, rt.date',
    sql_explanation => 'This query identifies multiple transactions for the same user that occur in different cities within a short time frame (e.g. one hour). This is a strong indicator of card-not-present fraud or that the card has been stolen or cloned.'
);
""")

sql_array.append(f"""
SELECT alloydb_ai_nl.add_template(
    nl_config_id => '{nl_config}',
    intent => 'Which types of transactions are most associated with fraud?',
    sql => 'SELECT mc.description, COUNT(t.id) AS fraudulent_transaction_count FROM transactions t JOIN fraud_labels fl ON t.id = fl.transaction_id JOIN mcc_codes mc ON t.mcc = mc.mcc WHERE fl.is_fraud = TRUE GROUP BY mc.description ORDER BY fraudulent_transaction_count DESC LIMIT 10',
    sql_explanation => 'Understanding which types of merchants are most frequently associated with fraudulent transactions can help in developing rules for fraud detection systems. This query counts the number of fraudulent transactions for each Merchant Category Code (MCC).'
);
""")

for sql in sql_array:
    result = await run_alloydb_query(finance_db_pool, sql)
    print(result)


In [ ]:
# View Created Templates
sql = f"""
SELECT id, nl, sql, intent, psql, pintent
FROM alloydb_ai_nl.template_store_view
WHERE config = '{nl_config}'
"""
await run_alloydb_query(finance_db_pool, sql)

You can optionally drop examples with a query like the following (pass the template ID as the only parameter):

```
SELECT alloydb_ai_nl.drop_template(1)
```

### Add Permissions for `toolbox_user` Database User

The AlloyDB Natural Language feature you just configured will be used by the `toolbox_user` account via MCP Toolbox, so we will add the necessary permissions for that user now.

In [ ]:
sql_array = []

sql_array.append(f"""
-- Grant usage on the alloydb_ai_nl schema
GRANT USAGE ON SCHEMA alloydb_ai_nl TO toolbox_user;
""")

sql_array.append(f"""
-- Grant select (read) access on all existing tables in the alloydb_ai_nl schema
GRANT SELECT ON ALL TABLES IN SCHEMA alloydb_ai_nl TO toolbox_user;
""")

sql_array.append(f"""
-- Grant execute permissions on all functions in the alloydb_ai_nl schema.
GRANT EXECUTE ON ALL FUNCTIONS IN SCHEMA alloydb_ai_nl TO toolbox_user;
""")

sql_array.append(f"""
-- Grant usage on all sequences in the alloydb_ai_nl schema
GRANT USAGE ON ALL SEQUENCES IN SCHEMA alloydb_ai_nl TO toolbox_user;
""")

sql_array.append(f"""
-- Ensure the user gets read access to any new tables created in the future
ALTER DEFAULT PRIVILEGES IN SCHEMA alloydb_ai_nl GRANT SELECT ON TABLES TO toolbox_user;
""")

sql_array.append(f"""
-- Grant usage on the google_ml schema
GRANT USAGE ON SCHEMA google_ml TO toolbox_user;
""")

sql_array.append(f"""
-- Grant select (read) access on all existing tables in the google_ml schema
GRANT SELECT ON ALL TABLES IN SCHEMA google_ml TO toolbox_user;
""")

sql_array.append(f"""
-- Grant execute permissions on all functions in the google_ml schema.
GRANT EXECUTE ON ALL FUNCTIONS IN SCHEMA google_ml TO toolbox_user;
""")

sql_array.append(f"""
-- Grant usage on all sequences in the google_ml schema
GRANT USAGE ON ALL SEQUENCES IN SCHEMA alloydb_ai_nl TO toolbox_user;
""")

sql_array.append(f"""
-- Ensure the user gets read access to any new tables created in the future
ALTER DEFAULT PRIVILEGES IN SCHEMA google_ml GRANT SELECT ON TABLES TO toolbox_user;
""")

for sql in sql_array:
    result = await run_alloydb_query(finance_db_pool, sql)
    print(result)


## Generate and Execute SQL with AlloyDB Natural Language

### Generate SQL from Natural Language Question

In [ ]:
sql = f"""SELECT
    alloydb_ai_nl.get_sql(
        '{nl_config}',
        'What is the total amount of transactions from the last 10 years?'
    ) ->> 'sql';"""
result = await run_alloydb_query(finance_db_pool, sql, output_as_df = False)
print(result.fetchall())

### Execute SQL from Natural Language Question

In [ ]:
sql = f"""SELECT
    alloydb_ai_nl.execute_nl_query(
        '{nl_config}',
        'What is the total amount of transactions from the last 10 years?'
    )"""
await run_alloydb_query(finance_db_pool, sql)

## Use AlloyDB Natural Language with MCP Toolbox

MCP Toolbox has [built-in](https://googleapis.github.io/genai-toolbox/resources/tools/alloydbainl/alloydb-ai-nl/) support for AlloyDB's Natural Language feature. To demonstrate the flexibility of this feature, let's deploy a 1-tool version of our Toolbox instance and ask a number of questions about the transactions, cards, users, mcc codes, and fraud labels in the database.

### Update the `tools.yaml` File

The `alloydb-ai-nl` tool type allows the agent to use the AlloyDB Natural Language feature to convert natural language questions into SQL queries.

In [ ]:
# Reference: https://googleapis.github.io/genai-toolbox/resources/sources/spanner/
#            https://googleapis.github.io/genai-toolbox/resources/tools/
#            https://googleapis.github.io/genai-toolbox/resources/tools/spanner-sql/
#            https://googleapis.github.io/genai-toolbox/resources/sources/alloydb-pg/
#            https://googleapis.github.io/genai-toolbox/resources/tools/postgres-sql/

import os
import json

tools_config = {
  "sources": {
    "alloydb-finance-source": {
      "kind": "alloydb-postgres",
      "project": f"{project_id}",
      "region": f"{region}",
      "cluster": f"{alloydb_cluster}",
      "instance": f"{alloydb_instance}",
      "database": f"{alloydb_database}",
      "user": "toolbox_user",
      "password": "${ALLOYDB_PASSWORD}",
      "ipType": "private"
    }
  },
  "tools": {
    "get_finance_database_context": {
      "kind": "alloydb-ai-nl",
      "source": "alloydb-finance-source",
      "description": "Use this tool to look up information about financial transactions, credit cards, customers, mcc codes, and historical fraud labels.",
      "nlConfig": f"{nl_config}"
    }
  },
  "toolsets": {
    "finance-toolset": [
      "get_finance_database_context",
    ]
  }
}

with open("tools.yaml", "w") as file:
    file.write(json.dumps(tools_config))


### Write Updated `tools.yaml` to Secret Manager

In [ ]:
# Create the secret
! gcloud secrets versions add tools --data-file=tools.yaml

In [ ]:
# Clean up the local file
import os
os.remove('tools.yaml')

### Update Toolbox with New `tools.yaml` File

In [ ]:
# Reference: https://cloud.google.com/sdk/gcloud/reference/run/services/update

! gcloud run services update toolbox --no-user-output-enabled \
    --update-secrets="/app/tools.yaml=tools:latest,ALLOYDB_PASSWORD=alloydb-password:latest" \
    --region={region}

### Execute Tool

In [ ]:
# Define a list of natural language questions to ask the MCP Toolbox tool.
nl_questions = [
    "How many users are in the database?",
    "Show me the most recent transactions for customer 123",
    "Tell me about the most recent fraudulent transactions",
    "Which customer has the most credit cards, and how many do they have?",
    "What are the top 10 highest salaries for our customers?",
    "Which mcc codes are most commonly associated with fraud?"
]

In [ ]:
import json
from toolbox_core import ToolboxClient, auth_methods
import urllib

import google.auth.transport.requests
import google.oauth2.id_token


def get_auth_token(endpoint):
    # Cloud Run uses your service's hostname as the `audience` value
    # audience = 'https://my-cloud-run-service.run.app/'
    # For Cloud Run, `endpoint` is the URL (hostname + path) receiving the request
    # endpoint = 'https://my-cloud-run-service.run.app/my/awesome/url'
    
    auth_req = google.auth.transport.requests.Request()
    id_token = google.oauth2.id_token.fetch_id_token(auth_req, endpoint)

    return id_token

# Get endpoint
toolbox_url = ! gcloud run services describe toolbox --region {region} --format 'value(metadata.annotations."run.googleapis.com/urls")'
toolbox_url = json.loads(toolbox_url[0])[0]
print(f"Toolbox Cloud Run endpoint: {toolbox_url}")

# Get auth_token
auth_token = get_auth_token(toolbox_url)
auth_token_provider = auth_methods.aget_google_id_token # can also use sync method

# Run tools 
async with ToolboxClient(
    toolbox_url,
    client_headers={"Authorization": f"Bearer {auth_token}"},
) as client:
    tools = await client.load_toolset("finance-toolset")
    for t in tools:
        print(f"\nRunning tool: {t._ToolboxTool__url}")
        for q in nl_questions:
            print(f"Question: {q}")
            result = await t(q)
            json_result = json.loads(result)
            print("Tool result:\n")
            print(json.dumps(json_result, indent=2))


## Use AlloyDB Natural Language with the ADK Agent

Now let's see our Finance Agent in action with this new flexible and powerful semantic query tool. 

### Update the Agent with New Tools

Updating the TOOLBOX_URL environment variable will force the ADK Agent to grab the new Tool we defined in the previous steps. All other environment variables will remain the same.

In [ ]:
! gcloud run services update finance-agent --no-user-output-enabled \
    --update-env-vars=TOOLBOX_URL={toolbox_url} \
    --region={region}

### Define Functions for Agent Invocation

In [ ]:
import requests
import json
import uuid
import google.auth
import google.auth.transport.requests
import google.oauth2.id_token

# --- Session Tracking ---
# Use a set to keep track of sessions created during this notebook's execution.
created_sessions = set()

def create_agent_session(agent_url: str, auth_token: str, app_name: str, user_id: str, session_id: str) -> bool:
    """
    Explicitly creates a new session for the agent.
    Returns True if successful, False otherwise.
    """
    session_url = f"{agent_url}/apps/{app_name}/users/{user_id}/sessions/{session_id}"
    headers = {
        "Authorization": f"Bearer {auth_token}",
        "Content-Type": "application/json"
    }
    payload = {"state": {}}

    try:
        response = requests.post(session_url, headers=headers, json=payload, timeout=60)
        response.raise_for_status()
        print(f"Successfully created session: {session_id}")
        return True
    except requests.exceptions.RequestException as e:
        print(f"\nError creating session: {e}")
        if e.response is not None:
            print(f"Response Status: {e.response.status_code}")
            print(f"Response Text: {e.response.text}")
        return False


def invoke_agent_streaming(query: str, session_id: str = None, user_id: str = 'default-user') -> None:
    """
    Invokes the agent on Cloud Run, creating a session only if it's new.
    """
    if session_id is None:
        session_id = str(uuid.uuid4())
    
    app_name = "finance_agent"

    try:
        agent_url = !gcloud run services describe finance-agent --region {region} --format 'value(status.url)'
        agent_url = agent_url[0]
        agent_auth_token = get_auth_token(agent_url)

        # --- Step 1: Create Session ONLY IF IT'S NEW ---
        if session_id not in created_sessions:
            if create_agent_session(agent_url, agent_auth_token, app_name, user_id, session_id):
                # Add the new session_id to our set of tracked sessions
                created_sessions.add(session_id)
            else:
                return # Stop if session creation fails

        # --- Step 2: Send the Message ---
        invoke_url = f"{agent_url}/run_sse"
        headers = {
            "Accept": "text/event-stream",
            "Authorization": f"Bearer {agent_auth_token}",
            "Content-Type": "application/json"
        }
        payload = {
            "app_name": app_name,
            "session_id": session_id,
            "user_id": user_id,
            "new_message": {
                "role": "user",
                "parts": [{"text": query}]
            }
        }

        with requests.post(invoke_url, headers=headers, json=payload, stream=True, timeout=300) as response:
            response.raise_for_status()
            print(f"\nUser Query: {query}")
            print("Agent Response:")
            for line in response.iter_lines():
                if line and line.decode('utf-8').startswith('data: '):
                    try:
                        json_data = json.loads(line.decode('utf-8')[6:])
                        text = json_data.get('content', {}).get('parts', [{}])[0].get('text', '')
                        print(text, end='', flush=True)
                    except (json.JSONDecodeError, IndexError):
                        continue
            print("\n" + "="*50 + "\n")

    except requests.exceptions.RequestException as e:
        print(f"\nError invoking agent: {e}")
        if e.response is not None:
            print(f"Response Status: {e.response.status_code}")
            print(f"Response Text: {e.response.text}")




### Query the Agent

We'll execute a multi-turn conversation with different natural language questions to watch how our ADK Agent uses the new AlloyDB Natural Language tool. 

In [ ]:
nl_questions = [
    "How many users are in the database?",
    "Show me the most recent transactions for customer 123",
    "Tell me about the most recent fraudulent transactions",
    "Which customer has the most credit cards, and how many do they have?",
    "What are the top 10 highest salaries for our customers?",
    "Which mcc codes are most commonly associated with fraud?"
]

# Set a unique ID for the conversation and user
conversation_session_id = f"session_{uuid.uuid4()}"
conversation_user_id = f"user_{uuid.uuid4()}"

for q in nl_questions:

    # The first call will see the session_id is new, create it, and add it to the 'created_sessions' set.
    invoke_agent_streaming(
        q,
        session_id=conversation_session_id,
        user_id=conversation_user_id
    )

Congratulations, you have completed Module 4! Proceed to [`5_build_an_interactive_finance_agent.ipynb`](./5_build_an_interactive_finance_agent.ipynb) to leverage all of the components you've built so far to build a power interactive ADK agent to assist Fraud Analysts with their fraud investigations.